In [1]:
import pandas as pd

import pyutils as u

/Users/mjain/DoSomething/data/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


https://trello.com/c/GLP95soX/1614-data-request-nfl-huddles

### Users signing up

In [2]:
sql_signups = '''
    SELECT s.campaign_id, s.northstar_id, u.state, u.city
	FROM signups s
	LEFT JOIN users u ON u.northstar_id = s.northstar_id
	WHERE s.campaign_id in ('9011', '9024', '9030', '9034') '''
df_signups = pd.read_sql(sql_signups, u.load_connection())

In [3]:
# clean state values and remove users who did not provide this information
df_signups = df_signups[~(df_signups['state'].isnull())]
df_signups = u.clean_state_data(df_signups)

In [4]:
# determines number of users per state and copies to clipboard
df_signups.groupby(['state']).agg({'northstar_id': lambda x: len(x.unique())}).to_clipboard()

In [5]:
# determines number of users per state and campaign and copies to clipboard
df_signups.groupby(['state', 'campaign_id']).agg({'northstar_id': lambda x: len(x.unique())}).unstack().to_clipboard()

In [6]:
# clean city values and remove users who did not provide this information
df_signups_cities = df_signups[df_signups['city'].notnull()]
df_signups_cities['city'] = df_signups_cities['city'].apply(lambda x: x.lower())
df_signups_cities['city'] = df_signups_cities['city'].apply(lambda x: x.strip())

/Users/mjain/DoSomething/data/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mjain/DoSomething/data/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
# remove duplicate city, state values to condense as much as possible to unique pairs
df_signups_cities[~(df_signups_cities[['city', 'state']].duplicated(keep='first'))][['city', 'state']].to_clipboard()

### Users reporting back
Same process as above.

In [8]:
sql_rbs = '''
    SELECT s.campaign_id, s.northstar_id, u.state, u.city
	FROM signups s
	LEFT JOIN users u ON u.northstar_id = s.northstar_id
	JOIN reportbacks r ON r.signup_id = s.id
	WHERE s.campaign_id in ('9011', '9024', '9030', '9034') '''
df_rbs = pd.read_sql(sql_rbs, u.load_connection())

In [9]:
df_rbs = df_rbs[~(df_rbs['state'].isnull())]
df_rbs = u.clean_state_data(df_rbs)

In [10]:
df_rbs.groupby(['state']).agg({'northstar_id': lambda x: len(x.unique())}).to_clipboard()

In [11]:
df_rbs.groupby(['state', 'campaign_id']).agg({'northstar_id': lambda x: len(x.unique())}).unstack().to_clipboard()

In [12]:
df_cities_rbs = df_rbs[df_rbs['city'].notnull()]
df_cities_rbs['city'] = df_cities_rbs['city'].apply(lambda x: x.lower())
df_cities_rbs['city'] = df_cities_rbs['city'].apply(lambda x: x.strip())

/Users/mjain/DoSomething/data/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mjain/DoSomething/data/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
df_cities_rbs[~(df_cities_rbs[['city', 'state']].duplicated(keep='first'))][['city', 'state']].to_clipboard()

## Gender breakdown

In [14]:
gender_sql = '''SELECT s.campaign_id, s.northstar_id, r.post_id, u.first_name, v.gender 
    FROM signups s 
    LEFT JOIN users u ON u.northstar_id = s.northstar_id
    LEFT JOIN voter_reg_gender_list v ON LOWER(u.first_name) = LOWER(v.name)
    LEFT JOIN reportbacks r ON r.signup_id = s.id
    WHERE s.campaign_id in ('9011', '9024', '9030', '9034') '''
df_gender = pd.read_sql(gender_sql, u.load_connection())

In [15]:
# remove users whose gender could not be determined
df_gender = df_gender[~df_gender['gender'].isnull()]

#### Users signing up

In [16]:
# determines number of users for each gender and copies to clipboard
df_gender.groupby(['gender']).agg({'northstar_id': lambda x: len(x.unique())}).to_clipboard()

In [17]:
# determines number of users for each gender and campaign and copies to clipboard
df_gender.groupby(['gender', 'campaign_id']).agg({'northstar_id': lambda x: len(x.unique())}).unstack().to_clipboard()

#### Users reporting back

In [18]:
df_gender_rbs = df_gender[df_gender['post_id'].notnull()]

In [19]:
df_gender_rbs.groupby(['gender']).agg({'northstar_id': lambda x: len(x.unique())}).to_clipboard()

In [20]:
df_gender_rbs.groupby(
    ['gender', 'campaign_id']).agg({'northstar_id': lambda x: len(x.unique())}).unstack().to_clipboard()